In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import shutil 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [8]:
def mkdir(path):
    isExists = os.path.exists(path) 
    if not isExists:
        os.makedirs(path)
        return
    else:
        shutil.rmtree(path)
        os.makedirs(path)
        return

In [9]:
def combine_data_C(data, C):
    data_with_c = data.copy()
    data_with_c['C'] = C
    return data_with_c

In [10]:
def nor_data(data2, method):
    if method == 0:
        return (data2 - data2.min())/(data2.max() - data2.min()) ##rescaling
    if method == 1:
        return (data2 - data2.mean())/(data2.max() - data2.min()) ##mean normalization
    if method == 2:
        return (data2-data2.mean())/(data2.std()) ##standarization

In [15]:
def csv_group(inputfile, clusternumber, plotpath = "./outplot", method = 2, groupkey = "indtype", outcsvpath = "./outcsv/", SaveEachPlot = False):
    ###
    # 1. Read data as dataframe ###
    #
    data = pd.read_csv(inputfile)

    ###
    # 2.Get various keyvalue ###
    #
    data_to_count = data[groupkey]
    intdic = {}
    for i in data_to_count: #count appe time
        if i in intdic:
            intdic[i] += 1
        else:
            intdic[i] = 1

    intlist = [] #choose desire key
    for i in intdic:
        if intdic[i] >= clusternumber: 
            intlist.append(i)
    intlist.sort() 
    #print(intlist)  ##intlist, len = 523 - 4, dtype = int

    ###
    # 3. Write intlist to csv file
    indtype_df = pd.DataFrame({'avail_ind':intlist})
    indtype_df.to_csv("availible_indtype.csv")

    ###
    # 4. Make dir for outplot
    if (SaveEachPlot):
        plotpath += "/c" + str(clusternumber) + "_m" + str(method) #make a new dir
        mkdir(plotpath)
        plotpath +=  "/ind="
    
    ###
    # 5. Data Process
    pdfpath = inputfile + "_c" + str(clusternumber) + "_m" + str(method) + ".pdf"
    with PdfPages(pdfpath) as pdf:
        num_int = len(intlist)
        for i in range(num_int):
            intkey = intlist[i]
            ##5.1 Get individual Data according to indtype
            data_to_processed = data[data[groupkey] == intkey]
            

            ##5.2 Process small data
            data2 = pd.DataFrame(data_to_processed, columns = ['rndinten', 'expinten'])
            ss = StandardScaler()
            data2 = ss.fit_transform(data2)

            ##5.3
            sk_kmeans = KMeans(n_clusters=clusternumber)
            sk_kmeans.fit(data2)

            ##5.4produce cluster label
            sk_C = sk_kmeans.predict(data2)
            data_with_c = combine_data_C(data_to_processed, sk_C)

            ##5.5 Save Plot
            sns.lmplot('rndinten', 'expinten', hue='C', data=data_with_c, fit_reg=False)
            plt.title("indtype = "+str(intkey))
            plt.tight_layout()
            pdf.savefig()
            if (SaveEachPlot):
                plt.savefig(plotpath + str(intkey)+".png", dpi = 200)
            plt.close()
    return 

In [16]:
infile = "2001_industry.csv"
num = 2
csv_group(infile, num)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)


In [ ]:
! git add .
! git commit -m "2.3update1"
! git push